# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
# Импорт библиотек
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
#знакомимся с данными
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
#проверяем на дубликаты
df.duplicated().sum()

0

In [6]:
#смотрим пропущенные значения
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Ознакомились с данными.В данных отсутствуют дубликаты и пропущенные значения. Можно переходить к следующему этапу.

## Разбейте данные на выборки

Разобъем выборку в пропорции 60/20/20

In [8]:
train_df, vnt_df = train_test_split(df, test_size=0.40, random_state=123456)
valid_df, test_df = train_test_split(vnt_df, test_size=0.50, random_state=123456)

print('Размер обучающей выборки', train_df.shape[0])
print('Размер валидационной выборки', valid_df.shape[0])
print('Размер тестовой выборки', test_df.shape[0])

Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


Выделим целевые признаки

In [9]:
train_df_features = train_df.drop(['is_ultra'], axis=1)
train_df_target = train_df['is_ultra']
valid_df_features = valid_df.drop(['is_ultra'], axis=1)
valid_df_target = valid_df['is_ultra']
test_df_features = test_df.drop(['is_ultra'], axis=1)
test_df_target = test_df['is_ultra']

**Вывод**

Данные были разбиты по пропорции: 60 - трейн / 20 - валидация / 20 - тест

Обучение пройдет на данных train_df

Подбор гиперпараметров на данных valid_df

Лучшая модель по валидации будет применена на данных test_df

## Исследуйте модели

**Tree classifier**

In [10]:
best_depth = 0
best_accuracy = 0

for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=123456, max_depth=depth)
    model.fit(train_df_features, train_df_target)
    valid_predictions = model.predict(valid_df_features)
    accuracy = accuracy_score(valid_df_target, valid_predictions)
    print('Глубина дерева', depth,'Точность', accuracy)
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy

print('Лучшая глубина дерева', best_depth,'Лучшая точность', best_accuracy)   

Глубина дерева 1 Точность 0.7309486780715396
Глубина дерева 2 Точность 0.7713841368584758
Глубина дерева 3 Точность 0.7916018662519441
Глубина дерева 4 Точность 0.7900466562986003
Глубина дерева 5 Точность 0.7978227060653188
Глубина дерева 6 Точность 0.7916018662519441
Глубина дерева 7 Точность 0.7822706065318819
Глубина дерева 8 Точность 0.7900466562986003
Глубина дерева 9 Точность 0.7853810264385692
Глубина дерева 10 Точность 0.7713841368584758
Глубина дерева 11 Точность 0.7807153965785381
Глубина дерева 12 Точность 0.76049766718507
Глубина дерева 13 Точность 0.7558320373250389
Глубина дерева 14 Точность 0.7402799377916018
Глубина дерева 15 Точность 0.7325038880248833
Глубина дерева 16 Точность 0.7262830482115086
Глубина дерева 17 Точность 0.7340590979782271
Глубина дерева 18 Точность 0.7200622083981337
Глубина дерева 19 Точность 0.7153965785381027
Лучшая глубина дерева 5 Лучшая точность 0.7978227060653188


**Randomforest**

In [11]:
best_accuracy = 0
best_depth = 0
for depth in range(1,20):
    for est in range(5,50,5):
        for sample in range(2,8):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, min_samples_leaf=sample, random_state=123456)
            model.fit(train_df_features, train_df_target)
            accuracy = model.score(valid_df_features, valid_df_target)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print(print("Глубина дерева:", best_depth, "    Количество деревьев:", best_est, "минимум объектов в узле:", best_sample, "  Качество:", best_accuracy))    

Глубина дерева: 7     Количество деревьев: 35 минимум объектов в узле: 2   Качество: 0.8180404354587869
None


**Logistic Regression**

In [12]:
model = LogisticRegression(random_state=123456)
model.fit(train_df_features, train_df_target)
accuracy = model.score(valid_df_features, valid_df_target)
print("Accuracy модели на валидационной выборке:", accuracy)

Accuracy модели на валидационной выборке: 0.6842923794712286


**Вывод**

  В ходе исследования были получены следующие данные:

* Точность модели решающего дерева при глубине 5 равна 0.7978227060653188

* Точность модели случайного леса при количестве деревьев 7 равна  0.8180404354587869

* Точность модели логистической регресии равна 0.6842923794712286 

* Учитывая, что выборка небольшая, то лучший результат показывает модель случайного леса

## Проверьте модель на тестовой выборке

Оценим точность всех моделей на тестовой выборке

**Tree classifier**

In [13]:
model = DecisionTreeClassifier(max_depth=5, random_state=123456)
model.fit(train_df_features, train_df_target)
accuracy = model.score(test_df_features, test_df_target)
print("Качество:", accuracy)

Качество: 0.8087091757387247


**Randomforest**

In [14]:
model = RandomForestClassifier(max_depth=7, n_estimators=35, min_samples_leaf=2, random_state=123456)
model.fit(train_df_features, train_df_target)
accuracy = model.score(test_df_features, test_df_target)
print("Качество:", accuracy)

Качество: 0.8133748055987559


**Logistic Regression**

In [15]:
model = LogisticRegression(random_state=123456)
model.fit(train_df_features, train_df_target)
accuracy = model.score(test_df_features, test_df_target)
print("Качество:", accuracy)

Качество: 0.7278382581648523


**Итоговый вывод**


В ходе проделанной работы:

Изучены данные.

Разбиты данные на выборки: обучающая, валидационная и тестовая.

Исследованы модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.

Найдены оптимальные параметры для каждой модели.

Оценена точность обученной модели.

Сделан вывод о том,  что лучшая модель - Randomforest
